In [1]:
import matplotlib as mpl
mpl.rcParams["figure.facecolor"] = "white"

mpl.rcParams["axes.facecolor"] = "white"
mpl.rcParams["savefig.facecolor"] = "white"
%matplotlib inline

In [2]:

import pandas as pd
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from calibration_dataset import Tell1Dataset

class MyDS(Tell1Dataset):
	filename_format = '%Y-%m-%d'
	filename_regex_format = r'\d{4}-\d{2}-\d{2}.csv'

datapath = "../../data/calibrations/"
data_list = MyDS.get_filepaths_from_dir(datapath)
mds = MyDS(data_list, read=True)



100%|██████████| 30/30 [00:07<00:00,  3.85it/s]


In [4]:
%load_ext tensorboard

In [5]:
from networks import VeloDecoderSmall, VeloEncoderSmall, VeloAutoencoderLt
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader, TensorDataset

In [10]:
mds.dfh['R'].df

,type,sensor,datetime,Zmod,slot_label,mod_nr,mod_type,sensor_type,sensor_number,channel0,...,channel2038,channel2039,channel2040,channel2041,channel2042,channel2043,channel2044,channel2045,channel2046,channel2047
2,hit_threshold,0.0,2010-08-16,-175,VL01L,mod_0,VELO_phi,R,#0,17.0,...,13.0,12.0,12.0,13.0,12.0,13.0,12.0,12.0,12.0,13.0
5,hit_threshold,0.0,2010-08-18,-175,VL01L,mod_0,VELO_phi,R,#0,17.0,...,12.0,12.0,12.0,12.0,13.0,13.0,12.0,13.0,12.0,12.0
8,hit_threshold,0.0,2011-03-07,-175,VL01L,mod_0,VELO_phi,R,#0,18.0,...,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0
11,hit_threshold,0.0,2011-05-01,-175,VL01L,mod_0,VELO_phi,R,#0,16.0,...,13.0,14.0,14.0,13.0,13.0,13.0,13.0,13.0,14.0,13.0
14,hit_threshold,0.0,2011-06-18,-175,VL01L,mod_0,VELO_phi,R,#0,10.0,...,13.0,13.0,13.0,13.0,13.0,13.0,14.0,14.0,13.0,14.0
17,hit_threshold,0.0,2011-07-26,-175,VL01L,mod_0,VELO_phi,R,#0,16.0,...,13.0,13.0,13.0,14.0,13.0,13.0,13.0,13.0,14.0,14.0
20,hit_threshold,0.0,2011-09-09,-175,VL01L,mod_0,VELO_phi,R,#0,17.0,...,14.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,14.0,14.0
23,hit_threshold,0.0,2012-04-06,-175,VL01L,mod_0,VELO_phi,R,#0,17.0,...,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0
26,hit_threshold,0.0,2012-04-27,-175,VL01L,mod_0,VELO_phi,R,#0,13.0,...,13.0,13.0,13.0,13.0,12.0,12.0,13.0,13.0,13.0,13.0
29,hit_threshold,0.0,2012-05-04,-175,VL01L,mod_0,VELO_phi,R,#0,17.0,...,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0


In [6]:
dfh = mds.dfh.df.iloc[:,9:]
dfh_r = mds.dfh['R'].df.iloc[:,9:]
dfh_phi = mds.dfh['phi'].df.iloc[:,9:]

In [7]:
dfp = mds.dfp.df.iloc[:,9:]
dfp_r = mds.dfp['R'].df.iloc[:,9:]
dfp_phi = mds.dfp['phi'].df.iloc[:,9:]

In [8]:
from sklearn.model_selection import train_test_split
from pytorch_lightning.loggers import TensorBoardLogger
import os



In [9]:
def make_loader(dataset):
    train, test = train_test_split(dataset, test_size=0.2)
    train_target = torch.tensor(train.values, dtype=torch.float)
    train_data = torch.tensor(train.values, dtype=torch.float)
    test_target = torch.tensor(test.values, dtype=torch.float)
    test_data = torch.tensor(test.values, dtype=torch.float)
    train_tensor = TensorDataset(train_data, train_target) 
    test_tensor = TensorDataset(test_data, test_target) 
    train_loader = DataLoader(dataset = train_tensor)
    test_loader = DataLoader(dataset = test_tensor)
    return train_loader, test_loader

In [10]:
from datetime import datetime
datetime.now().strftime("%d_%m_%Y_%H_%M_%S")

'22_10_2020_15_26_46'

In [11]:
def make_model_trainer(s, version):
    s = 2048
    dec = VeloDecoderSmall(s)
    enc = VeloEncoderSmall(s)
    model = VeloAutoencoderLt(enc, dec)

    logger = TensorBoardLogger(
        save_dir=os.getcwd(),
        version=version+datetime.now().strftime("_%d_%m_%Y_%H_%M_%S"),
        name='lightning_logs'
    )
    tr = pl.Trainer(logger=logger, max_epochs=50, gpus=1)
    return model, tr



In [12]:
def run_experiment(dataset, version):
    train_loader, test_loader = make_loader(dataset)
    s = dataset.shape[1]
    model, tr = make_model_trainer(s, version)
    tr.fit(model, train_loader, test_loader)

In [13]:
run_experiment(dfh, "dfh_small")


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | enc  | VeloEncoderSmall | 82 K  
1 | dec  | VeloDecoderSmall | 84 K  
/home/mwm/anaconda3/envs/velo_ai/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/mwm/anaconda3/envs/velo_ai/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


In [14]:
run_experiment(dfp, "dfp_small")
run_experiment(dfp_r, "dfp_r_small")
run_experiment(dfh_r, "dfh_r_small")
run_experiment(dfp_phi, "dfp_phi_small")
run_experiment(dfh_phi, "dfh_phi_small")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | enc  | VeloEncoderSmall | 82 K  
1 | dec  | VeloDecoderSmall | 84 K  


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | enc  | VeloEncoderSmall | 82 K  
1 | dec  | VeloDecoderSmall | 84 K  


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | enc  | VeloEncoderSmall | 82 K  
1 | dec  | VeloDecoderSmall | 84 K  


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | enc  | VeloEncoderSmall | 82 K  
1 | dec  | VeloDecoderSmall | 84 K  


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | enc  | VeloEncoderSmall | 82 K  
1 | dec  | VeloDecoderSmall | 84 K  


In [ ]:
%tensorboard --logdir lightning_logs --host 0.0.0.0


Launching TensorBoard...